In [2]:
# Insatll 
# 1. Meta/Facebook FAISS (Facebook AI Similarity Search) Package
# 2. Sentence Transformer package
#!conda install faiss-cpu
!conda install -y -c pytorch faiss-cpu   # Need Python verion 3.11
#!conda install sentence-transformers
!conda install -y -c conda-forge sentence-transformers

Channels:
 - pytorch
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/peter/anaconda3/envs/tf

  added / updated specs:
    - faiss-cpu


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2023.7.22~ --> pkgs/main/linux-64::certifi-2023.7.22-py310h06a4308_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Channels:
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/peter/anaconda3/envs/tf

  added / updated specs:
    - sentence-transformers


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            pkgs/main/linux-64::certifi-2023.7.22~ --> conda-forge/noarch::certifi-2023.7.22-pyhd8ed1ab_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
# import Pandas (Panel Data) row/column library
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

text  \
0                                        Meditation and yoga can improve mental health   
1                     Fruits, whole grains and vegetables helps control blood pressure   
2                                    These are the latest fashion trends for this week   
3                                    Vibrant color jeans for male are becoming a trend   
4                                                   The concert starts at 7 PM tonight   
5                      Navaratri dandiya program at Expo center in Mumbai this october   
6                                    Exciting vacation destinations for your next trip   
7  Maldives and Srilanka are gaining popularity in terms of low budget vacation places   

  category  
0   Health  
1   Health  
2  Fashion  
3  Fashion  
4    Event  
5    Event  
6   Travel  
7   Travel

### Step 1 : Create source embeddings for the text column

In [6]:
#!conda install transformers
from sentence_transformers import SentenceTransformer

In [7]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [8]:
print('vectors.shape:')
print(vectors.shape)
print()
print('df.head():')
print(df.head())
print()
print('vectors[:5]')
print(vectors[:5])

vectors.shape:
(8, 768)

df.head():
                                                               text category
0                     Meditation and yoga can improve mental health   Health
1  Fruits, whole grains and vegetables helps control blood pressure   Health
2                 These are the latest fashion trends for this week  Fashion
3                 Vibrant color jeans for male are becoming a trend  Fashion
4                                The concert starts at 7 PM tonight    Event

vectors[:5]
[[-0.00247387  0.03626728 -0.05290459 ... -0.09152357 -0.03970004
  -0.04330489]
 [-0.03357267  0.00980517 -0.03250133 ... -0.05165474  0.02245888
  -0.03156183]
 [-0.01865329 -0.04051307 -0.0123539  ...  0.00610585 -0.07179645
   0.02773847]
 [-0.00338735  0.07687491  0.01197136 ...  0.06366339 -0.02566213
   0.01387302]
 [-0.05906574 -0.03065013 -0.01328563 ... -0.02871671 -0.02685121
  -0.01339997]]


In [9]:
dim0 = vectors.shape[0]
dim1 = vectors.shape[1]
print('dim0:', dim0)
print('dim1:', dim1)

dim0: 8
dim1: 768


### Step 2 : Build a FAISS Index for vectors

In [10]:
import faiss
dim = vectors.shape[1]
index = faiss.IndexFlatL2(dim)
print("index:")
print(index)

index:
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fe5633bad60> >


### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [11]:
index.add(vectors)
print('index:')
print(index)

index:
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fe5633bad60> >


### Step 4 : Encode search text using same encorder and normalize the output vector

In [12]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
print('search_query:', search_query)
print('vec.shape:', vec.shape)
print('vec:', vec)

search_query: I want to buy a polo t-shirt
vec.shape: (768,)
vec: [ 1.03882765e-02  2.78685242e-02 -1.18618309e-02  1.81327015e-02
  1.21984328e-03 -1.42995575e-02  1.96229927e-02  2.07198039e-02
 -2.23660562e-02  4.75626662e-02  1.77976284e-02 -8.00336711e-03
  2.53419280e-02  5.26148193e-02  8.44943058e-03 -1.63943898e-02
  1.02661196e-02 -2.74856482e-02  8.37066919e-02 -1.52885541e-02
  1.67682339e-02 -3.97140533e-03 -2.74348911e-02  5.02092317e-02
 -8.36519618e-03 -4.74074744e-02  2.36915518e-02 -1.01186074e-02
 -2.82840244e-02  7.94265419e-03  4.21435349e-02 -4.19379584e-03
 -1.91232506e-02 -3.12785320e-02  1.24685050e-06 -1.04428595e-02
 -2.19971631e-02 -8.66928175e-02 -1.88500492e-03 -2.54772156e-02
 -9.72974859e-03  7.93310329e-02 -3.55963111e-02 -3.05347115e-04
 -1.12392465e-02 -3.88932489e-02  5.49314432e-02  1.35207951e-01
 -8.19147155e-02  1.18784793e-02 -9.01846308e-03  1.92544423e-02
 -2.82723531e-02 -3.29266898e-02 -2.20107660e-02 -4.29836139e-02
  3.70104499e-02 -4.8064

In [13]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
print('svec.shape:', svec.shape)
print('svec:', svec)

svec.shape: (1, 768)
svec: [[ 1.03882765e-02  2.78685242e-02 -1.18618309e-02  1.81327015e-02
   1.21984328e-03 -1.42995575e-02  1.96229927e-02  2.07198039e-02
  -2.23660562e-02  4.75626662e-02  1.77976284e-02 -8.00336711e-03
   2.53419280e-02  5.26148193e-02  8.44943058e-03 -1.63943898e-02
   1.02661196e-02 -2.74856482e-02  8.37066919e-02 -1.52885541e-02
   1.67682339e-02 -3.97140533e-03 -2.74348911e-02  5.02092317e-02
  -8.36519618e-03 -4.74074744e-02  2.36915518e-02 -1.01186074e-02
  -2.82840244e-02  7.94265419e-03  4.21435349e-02 -4.19379584e-03
  -1.91232506e-02 -3.12785320e-02  1.24685050e-06 -1.04428595e-02
  -2.19971631e-02 -8.66928175e-02 -1.88500492e-03 -2.54772156e-02
  -9.72974859e-03  7.93310329e-02 -3.55963111e-02 -3.05347115e-04
  -1.12392465e-02 -3.88932489e-02  5.49314432e-02  1.35207951e-01
  -8.19147155e-02  1.18784793e-02 -9.01846308e-03  1.92544423e-02
  -2.82723531e-02 -3.29266898e-02 -2.20107660e-02 -4.29836139e-02
   3.70104499e-02 -4.80649062e-02  9.49249417e-03

In [14]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [15]:
# We get the index of original dataset and distances.
distances, I = index.search(svec, k=2)
print('search_query:', search_query)
print('orginal daatset df:')
print(df)                         # orginal dataset
print()
print('index I:', I)              # [[3, 2]]
print('I.tolist():', I.tolist())  # [[3, 2]]
row_indices = I.tolist()[0]
print()
print('row_indices:', row_indices)
print('df.loc[row_indices]:', df.loc[row_indices])
print()
print('df.loc[I[0]]:', df.loc[I[0]]) # You can acces by I[0]
print()
print('distances:', distances)

search_query: I want to buy a polo t-shirt
orginal daatset df:
                                                                                  text  \
0                                        Meditation and yoga can improve mental health   
1                     Fruits, whole grains and vegetables helps control blood pressure   
2                                    These are the latest fashion trends for this week   
3                                    Vibrant color jeans for male are becoming a trend   
4                                                   The concert starts at 7 PM tonight   
5                      Navaratri dandiya program at Expo center in Mumbai this october   
6                                    Exciting vacation destinations for your next trip   
7  Maldives and Srilanka are gaining popularity in terms of low budget vacation places   

  category  
0   Health  
1   Health  
2  Fashion  
3  Fashion  
4    Event  
5    Event  
6   Travel  
7   Travel  

index I:

In [16]:
I

array([[3, 2]])

In [17]:
I.tolist()

[[3, 2]]

In [18]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [19]:
df.loc[row_indices]

text category
3  Vibrant color jeans for male are becoming a trend  Fashion
2  These are the latest fashion trends for this week  Fashion

In [20]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query